# Introduction

In this notebook, we will perform an analysis of iceberg dataset. The goal is to strike the differences between the two labels (ship or iceberg).

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [ ]:
train = pd.read_json('../input/train.json')

#### Get the indices of 'ship' and 'iceberg'

In [ ]:
index_ship=np.where(train['is_iceberg']==0)
index_ice=np.where(train['is_iceberg']==1)

#### Function to plot the images

In [ ]:
def plots(band,index,title):
    plt.figure(figsize=(12,10))
    for i in range(12):
        plt.subplot(3,4,i+1)
        plt.xticks(())
        plt.yticks(())
        plt.xlabel((title))
        plt.imshow(np.reshape(train[band][index[0][i]], (75,75)),cmap='gist_heat')
    plt.show()  

#### Images of band 1 ships

In [ ]:
plots('band_1',index_ship,'band1 ship')

#### Images of band 2 ships

In [ ]:
plots('band_2',index_ship,'band2 ship')

#### Images of band 1 iceberg

In [ ]:
plots('band_1',index_ice,'band1 ice')

#### Images of band 2 icebergs

In [ ]:
plots('band_2',index_ice,'band2 ice')

#### Observations: 
 As seen from the above plots, we have uniform intensity (representing the 'clutter') images with impulse change in intensity at the center (representing the 'ship' or 'iceberg'). Such a distrupt change in the intensity is reflected by the edges. So, we use any standard edge detection techniques can be used to segment these images. Here, we will try three different operators:
1. Sobel Operator
2. Roberts Operator
3. Prewitt Operator

In [ ]:
def plot_conv(band,index,xFilt,yFilt,title):
    plt.figure(figsize=(12,10))
    for i in range(12):
        plt.subplot(3,4,i+1)
        plt.xticks(())
        plt.yticks(())
        plt.xlabel((title))
        img=np.reshape(train[band][index[0][i]], (75,75))
        img_Gx=signal.convolve2d(img,xFilt,mode='valid')
        img_Gy=signal.convolve2d(img,yFilt,mode='valid')
        img_filt=np.hypot(img_Gx,img_Gy)

        plt.imshow((img_filt) ,cmap='gist_heat')
    plt.show()
    

#### Sobel operator for band 1 iceberg

In [ ]:
##sobel operator

Gx_sobel=np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
Gy_sobel=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
lpf=1.0/25*np.ones((5,5))
plot_conv('band_1', index_ice,Gx_sobel,Gy_sobel,'band1 ice using sobel')

#### Sobel operator for band 2 iceberg

In [ ]:
plot_conv('band_2', index_ice,Gx_sobel,Gy_sobel,'band2 ice using sobel')

#### Sobel operator for band 1 iship

In [ ]:
plot_conv('band_1', index_ship,Gx_sobel,Gy_sobel,'band1 ship using sobel')

#### Sobel operator for band 2 ship

In [ ]:
plot_conv('band_2', index_ship,Gx_sobel,Gy_sobel,'band2 ship using sobel')

#### Observations

As seen from above plots, the sobel operator is able to segment the images. However, in certain cases the clutter edges are also highlighted. To counter this, we can first use averaging filter before applyting edge detector. This will smoothens the edges of clutter. We will be using 5x5 averaging filter in our analysis.

In [ ]:
def plot_Avgconv(band,index,lpf,xFilt,yFilt,title):
    plt.figure(figsize=(12,10))
    for i in range(12):
        plt.subplot(3,4,i+1)
        plt.xticks(())
        plt.yticks(())
        plt.xlabel((title))
        img=np.reshape(train[band][index[0][i]], (75,75))
        img_lpf=signal.convolve2d(img,lpf,mode='valid')
        img_Gx=signal.convolve2d(img_lpf,xFilt,mode='valid')
        img_Gy=signal.convolve2d(img_lpf,yFilt,mode='valid')
        img_filt=np.hypot(img_Gx,img_Gy)

        plt.imshow((img_filt) ,cmap='gist_heat')
    plt.show()

#### averaging + sobel operator on band 1 iceberg

In [ ]:
##avg+sobel operator
plot_Avgconv('band_1', index_ice,lpf,Gx_sobel,Gy_sobel,'band1 ice using avg+sobel')

#### averaging + sobel operator on band 2 iceberg

In [ ]:
plot_Avgconv('band_2', index_ice,lpf,Gx_sobel,Gy_sobel,'band2 ice using avg+sobel')

#### averaging + sobel operator on band 1 ship

In [ ]:
plot_Avgconv('band_1', index_ship,lpf,Gx_sobel,Gy_sobel,'band1 ship using avg+sobel')

#### averaging + sobel operator on band 2 ship

In [ ]:
plot_Avgconv('band_2', index_ship,lpf,Gx_sobel,Gy_sobel,'band2 ship using avg+sobel')

#### Obsevations:

Averaging followed by sobel operator is able to highlight the object by suppressing the effect of clutter. Also, the edges in ship are defined by nice contours as compared to iceberg. This should come as no surprise because the structure like ship have well defined contours. 

#### Averaging + prewitt operator on band 1 ship

In [ ]:
##Prewitt operator + averaging

Gx_pre=np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
Gy_pre=np.array([[-1,-1,-1],[0,0,0],[1,1,1]])
plot_Avgconv('band_1', index_ship,lpf,Gx_pre,Gy_pre,'band1 ship using avg+prewitt')

#### Averaging + prewitt operator on band 1 iceberg

In [ ]:
plot_Avgconv('band_1', index_ice,lpf,Gx_pre,Gy_pre,'band1 ice using avg+prewitt')

#### Averaging + roberts operator on band 1 ship

In [ ]:
Gx_rob=np.array([[1,0],[0,-1]])
Gy_rob=np.array([[0,1],[-1,0]])
plot_Avgconv('band_1', index_ship,lpf,Gx_rob,Gy_rob,'band1 ship using avg+roberts')

#### Averaging + prewitt operator on band 1 iceberg

In [ ]:
plot_Avgconv('band_1', index_ice,lpf,Gx_rob,Gy_rob,'band1 ice using avg+roberts')

#### Comparison of all operators on band 1 iceberg

In [ ]:
plt.figure(figsize=(14,12))
k=0
for i in range(4):
    k+=1
    plt.subplot(4,3,k)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(('band1 ice using avg+sobel'))
    img=np.reshape(train['band_1'][index_ice[0][i]], (75,75))
    img_lpf=signal.convolve2d(img,lpf,mode='valid')
    img_Gx=signal.convolve2d(img_lpf,Gx_sobel,mode='valid')
    img_Gy=signal.convolve2d(img_lpf,Gy_sobel,mode='valid')
    img_filt=np.hypot(img_Gx,img_Gy)
  
    plt.imshow((img_filt) ,cmap='gist_heat')
                
    k+=1
    plt.subplot(4,3,k)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(('band1 ice using avg+prewitt'))
    img=np.reshape(train['band_1'][index_ice[0][i]], (75,75))
    img_lpf=signal.convolve2d(img,lpf,mode='valid')
    img_Gx=signal.convolve2d(img_lpf,Gx_pre,mode='valid')
    img_Gy=signal.convolve2d(img_lpf,Gy_pre,mode='valid')
    img_filt=np.hypot(img_Gx,img_Gy)
  
    plt.imshow((img_filt) ,cmap='gist_heat')
                
    k+=1
    plt.subplot(4,3,k)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(('band1 ice using avg+roberts'))
    img=np.reshape(train['band_1'][index_ice[0][i]], (75,75))
    img_lpf=signal.convolve2d(img,lpf,mode='valid')
    img_Gx=signal.convolve2d(img_lpf,Gx_rob,mode='valid')
    img_Gy=signal.convolve2d(img_lpf,Gy_rob,mode='valid')
    img_filt=np.hypot(img_Gx,img_Gy)
  
    plt.imshow((img_filt) ,cmap='gist_heat')
plt.show()

#### Comparison of all operators on band 1 ship

In [ ]:
plt.figure(figsize=(14,12))
k=0
for i in range(4):
    k+=1
    plt.subplot(4,3,k)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(('band1 ship using avg+sobel'))
    img=np.reshape(train['band_1'][index_ship[0][i]], (75,75))
    img_lpf=signal.convolve2d(img,lpf,mode='valid')
    img_Gx=signal.convolve2d(img_lpf,Gx_sobel,mode='valid')
    img_Gy=signal.convolve2d(img_lpf,Gy_sobel,mode='valid')
    img_filt=np.hypot(img_Gx,img_Gy)
  
    plt.imshow((img_filt) ,cmap='gist_heat')
                
    k+=1
    plt.subplot(4,3,k)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(('band1 ship using avg+prewitt'))
    img=np.reshape(train['band_1'][index_ship[0][i]], (75,75))
    img_lpf=signal.convolve2d(img,lpf,mode='valid')
    img_Gx=signal.convolve2d(img_lpf,Gx_pre,mode='valid')
    img_Gy=signal.convolve2d(img_lpf,Gy_pre,mode='valid')
    img_filt=np.hypot(img_Gx,img_Gy)
  
    plt.imshow((img_filt) ,cmap='gist_heat')
                
    k+=1
    plt.subplot(4,3,k)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(('band1 ship using avg+roberts'))
    img=np.reshape(train['band_1'][index_ship[0][i]], (75,75))
    img_lpf=signal.convolve2d(img,lpf,mode='valid')
    img_Gx=signal.convolve2d(img_lpf,Gx_rob,mode='valid')
    img_Gy=signal.convolve2d(img_lpf,Gy_rob,mode='valid')
    img_filt=np.hypot(img_Gx,img_Gy)
  
    plt.imshow((img_filt) ,cmap='gist_heat')
plt.show()

#### Conclusion

All three operators are able to segment the images (no surprise here). Qualitative, Sobel and Prewitt operators have same performance but Roberts operator is lagging behind a little in suppressing the clutter. 
